In [ ]:
# this notebook is for figuring out how strong the length correlations of the RMs
# we're using are

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from rlhfutils.debug_utils import load_all_dfs, load_rm, progress_rm

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import spearmanr, pearsonr
import random
import torch

In [ ]:
adfs = load_all_dfs("../outputs/lsanity/")

In [ ]:
mpaths = [
    "../stack-llama/models/rmodel/",
    "/home/prasann/Projects/tfr-decoding/trlx_train/trl-stack/models/rewardda/",
    "/home/prasann/Projects/tfr-decoding/trlx_train/trl-stack/models/rewardmixed/",
    "/home/prasann/Projects/tfr-decoding/trlx_train/trl-stack/models/rewarddalen/"
]
toker, rm, kwargs = load_rm(mpaths[3], 2)

In [ ]:
respset = []
lenset = []
for a in adfs:
    respset.extend(list(adfs[a]['response']))
    lenset.extend([len(toker(ans).input_ids) for ans in adfs[a]['answer']])
flim = 0
responses = []
lens = []
origlens = []
for i in range(len(respset)):
    if lenset[i]>450:
        trunclen = random.randint(0, 400)
        responses.append(toker.decode(toker(respset[i]).input_ids[:-trunclen]))
        lens.append(lenset[i] - trunclen)
        origlens.append(lenset[i])

In [ ]:
randinds = random.sample(list(range(len(responses))), 64)
lens = [lens[ind] for ind in randinds]
responses = [responses[ind] for ind in randinds]

In [ ]:
plt.hist(lens)

In [ ]:
torch.cuda.empty_cache()

In [ ]:
scos = progress_rm(responses, rm, kwargs)

In [ ]:
scos = [s[0]['score'] for s in scos]

In [ ]:
plt.scatter(lens, scos)

In [ ]:
print(spearmanr(scos, lens))
print(pearsonr(scos, lens))


In [ ]:
# check apfarm issue in a bit of detail